In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.display import clear_output

In [2]:
import pickle #for saving output files, pickles
import time #for time.sleep function to delay calls
from tqdm import tqdm #for updating loop
#from os import listdir
#from os.path import isfile, join
import glob #pattern matching and expansion.

In [93]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [4]:
import requests
import random
from itertools import cycle
import traceback

user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]


In [5]:
IP_start = pd.read_csv("/home/russell/Documents/DataScience/DonorsChoose/IPs/eighty_1.csv",header=None)
proxies = IP_start.iloc[:,0].values.tolist()
random.shuffle(proxies)
proxy_pool = cycle(proxies)

In [52]:
def proposal_process(proplist,requestpath):
    '''takes a list with proposal info and processes some info into a df '''
    #we can use values in these keys 'as is'
    keeperkeys = ['id', 'proposalURL', 'fundURL', 'imageURL', 'retinaImageURL', 'thumbImageURL', 'fulfillmentTrailer', 'percentFunded', 'numDonors', 'costToComplete', 'studentLed', 'numStudents', 'professionalDevelopment', 'distanceLearningProject', 'totalPrice', 'freeShipping', 'teacherId', 'teacherName', 'teacherTypes', 'schoolTypes', 'schoolName', 'schoolUrl', 'city', 'zip', 'state', 'stateFullName', 'latitude', 'longitude', 'expirationDate', 'expirationTime', 'fundingStatus', 'fullyFundedDate', 'waitingForCheckPayment', 'modifiedDate','shortDescription']
    
    #these require more finagling
    moreprocess = ['matchingFund', 'gradeLevel', 'povertyType']
    
    combine=[]
    
    for prop in proplist:
        new_dict = {k: prop[k] for k in keeperkeys if k in prop}
        keepinfo = pd.DataFrame.from_dict(new_dict,orient='index').T
        
        if(not bool(prop['matchingFund'].get('matchingKey'))):
            keepinfo['matchingfund'] = 'no'
        else:
            keepinfo['matchingfund'] = 'no'
            
        keepinfo['grade_level'] = prop['gradeLevel']['name']
        keepinfo['poverty_level'] = prop['povertyType']['label']
        combine.append(keepinfo)
        
    summarydf = pd.concat(combine)
    summarydf['r_path']=requestpath*summarydf.shape[0]
    return(summarydf)

In [166]:
#baseurl = 'https://www.donorschoose.org/donors/search.html?historical=true&searchTag=-3' #funded, classroom projects only (no PD)
baseurl = 'https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&historical=true&searchTag=-3'

states = ["","AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
addme = 'state='
statelist = [addme + s for s in states]

subjectlist = ["",'subject5=-5','subject5=32','subject5=16','subject5=21','subject5=18','subject5=17','subject5=19','subject5=20',
               'subject2=-2','subject2=29','subject2=27','subject2=28','subject2=2',
              'subject3=-3','subject3=13','subject3=14','subject3=9','subject3=15',
               'subject4=-4','subject4=6','subject4=7','subject4=30','subject4=4','subject4=8',
               'subject1=-1','subject1=12','subject1=1','subject1=11']
                    # 'subject7=-7','subject8=-8'] #ignore #specialneeds(subject7),warmthcarehunger(subject8)
#applied learning (subject5),health&sports (subject2), history&civics (subject3),literacy&lang(subject4),math&sci(subject1),


showonlylist = ['','metroType=r','teacherNotFunded=true','moderateHighEconomicNeed=true','distanceLearning=only']
#rural, teacher never funded before, high economic need, distance learning

#No High School requests
agegrouplist = ['','gradeType=1','gradeType=1&gradeLevel=-1','gradeType=1&gradeLevel=0','gradeType=1&gradeLevel=1','gradeType=1&gradeLevel=2',
                'gradeType=2','gradeType=2&gradeLevel=3','gradeType=2&gradeLevel=4','gradeType=2&gradeLevel=5',
                'gradeType=3','gradeType=3&gradeLevel=6','gradeType=3&gradeLevel=7','gradeType=3&gradeLevel=8',
                'gradeType=4','gradeType=4&gradeLevel=9','gradeType=4&gradeLevel=10','gradeType=4&gradeLevel=11','gradeType=4&gradeLevel=12']

#costlist = ['costToComplete=1','costToComplete=2',] #this has strange results for historical values

#no filters by class of requests (e.g. art supplies)
requestsforlist = ['']
#                   'proposalType=16','proposalTypeCombo=Books','proposalType=10','proposalType=7',
#                   'proposalType=15','proposalType=12','proposalType=18','proposalType=8',
#                   'proposalType=11','proposalType=17','proposalType=13','proposalType=14']


teacherfundedlist = ['','teacherNotFunded=true','teacherType=1'] #never funded teachers, TFA teachers
schooltypelist =['']
#schooltypelist = ['','schoolType=16','schoolType=1','schoolType=21','schoolType=5']#traditional, charter, head start, year-round
#'schoolType=24', serving military

In [167]:
baseurl


'https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&historical=true&searchTag=-3'

In [168]:
#HERE, we will loop through the lists created above, creating all possible combinations of the sub-categories
# EACH list has an empty element at the beginning, which we will handle in every loop
# by effectively leaving that category BLANK (i.e. not using that filter)
# AT THE END, we will add every new address to 'searchurls' (which we will then shuffle)


####NOTE----this code CAN use a lot of memory and generate >40 million combinations IF ALL lists above are 'full'
#          (often causing a crash if many other processes are running)

search_urls =[]

#loop through every state
for state in statelist:
    if state == '':
        combo_url = baseurl
    else:
        combo_url = baseurl+'&'+state
        
    for subject in subjectlist:
        if subject == '':
            new_combo_url = combo_url
        else:
            new_combo_url = combo_url+'&'+subject
            
        for showme in showonlylist:
            if showme == '':
                double_new_url = new_combo_url
            else:
                double_new_url = new_combo_url+'&'+showme
                
            for agegroup in agegrouplist:
                if agegroup == '':
                    triple_new_url = double_new_url
                else:
                    triple_new_url = double_new_url+'&'+agegroup
                    
                for reQuest in requestsforlist:
                    if reQuest == '':
                        quad_new_url = triple_new_url
                    else:
                        quad_new_url = triple_new_url+'&'+reQuest
                    
                    for tf in teacherfundedlist:
                        if tf == '':
                            quint_new_url = quad_new_url
                        else:
                            quint_new_url = quad_new_url+'&'+tf
                            
                        for schType in schooltypelist:
                            if schType == '':
                                sext_new_url = quint_new_url
                            else:
                                sext_new_url = quint_new_url+'&'+schType
                                
                            search_urls.append(sext_new_url)
                            

In [169]:
len(search_urls)

429780

In [170]:
import random

tenrandos=[]

for x in range(10):
    #print(search_urls[x])
    #print(random.choice(search_urls))
    tenrandos.append(random.choice(search_urls))



In [172]:
random.shuffle(search_urls) #shuffles *search_urls* so calls are bouncing around randomly

In [ ]:
prog_count = 0
iteration = 0
out_file = "/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_"

#for x in tenrandos:
for x in search_urls:  #1674610
        
    page_to_use = x
    
    #Make the request
    try:
        r = requests.get(page_to_use) #send request to specific html address
    except:
        #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
        continue #if we have an exception, *continue* will skip the rest of the iteration
    
    try:
        data_dict = json.loads(r.text) #load the text as a dictionary
    except: #catch *all* exceptions
        #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
        continue #if we have an exception, *continue* will skip the rest of the iteration
    
    dicts_on_this_page = list(data_dict.values())
    proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
    proposal_list = dicts_on_this_page[proposal_ind]

    #if projects are returned with these criteria, len(proposal_list) will be > 0
    if len(proposal_list)>0:
        
            outframe = proposal_process(proposal_list,page_to_use)
            #print(outframe['teacherName'])
            pickle_out = open(out_file+("{0:07}".format(iteration))+'.pickle',"wb")
            pickle.dump(outframe, pickle_out)
            pickle_out.close()
            iteration += 1
            prog_count += 50
            more_to_get = True
            indexup = 0
            
            while more_to_get:
                indexup +=50
                new_page_to_use = page_to_use+'&index='+str(indexup)
                try:
                    r = requests.get(new_page_to_use) #send request to specific html address
                except:
                    #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                    more_to_get = False
                    continue #if we have an exception, *continue* will skip the rest of the iteration

                try:
                    data_dict = json.loads(r.text) #load the text as a dictionary
                except: #catch *all* exceptions
                    #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                    more_to_get = False
                    continue #if we have an exception, *continue* will skip the rest of the iteration
                
                dicts_on_this_page = list(data_dict.values())
                proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
                proposal_list = dicts_on_this_page[proposal_ind]

                #if projects are returned with these criteria, len(proposal_list) will be > 0
                if len(proposal_list)>0:
                    outframe = proposal_process(proposal_list,page_to_use)
                    #print(outframe['teacherName'])
                    pickle_out = open(out_file+("{0:07}".format(iteration))+'.pickle',"wb")
                    pickle.dump(outframe, pickle_out)
                    pickle_out.close()
                    iteration += 1
                    prog_count += 50
                else:
                    more_to_get = False
                
    print("Current progress:",np.round(iteration/(len(tenrandos))),"%")
    time.sleep(np.random.rand()*5)

Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 1.0 %
Current progress: 1.0 %
Current progress: 1.0 %
Current progress: 1.0 %
Current progress: 1.0 %
Current progress: 1.0 %
Current progress: 1.0 %
Current progress: 1.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress: 2.0 %
Current progress

Current progress: 35.0 %
Current progress: 37.0 %
Current progress: 38.0 %
Current progress: 38.0 %
Current progress: 38.0 %
Current progress: 38.0 %
Current progress: 38.0 %
Current progress: 38.0 %
Current progress: 39.0 %
Current progress: 39.0 %
Current progress: 39.0 %
Current progress: 39.0 %
Current progress: 39.0 %
Current progress: 39.0 %
Current progress: 39.0 %
Current progress: 39.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 40.0 %
Current progress: 41.0 %
Current progress: 41.0 %
Current progress: 41.0 %
Current progress: 41.0 %
Current progress: 41.0 %
Current progress: 41.0 %
Current progress: 41.0 %
Current progress: 41.0 %


Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 75.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 76.0 %
Current progress: 77.0 %
Current progress: 77.0 %


Current progress: 101.0 %
Current progress: 101.0 %
Current progress: 101.0 %
Current progress: 101.0 %
Current progress: 101.0 %
Current progress: 102.0 %
Current progress: 102.0 %
Current progress: 102.0 %
Current progress: 102.0 %
Current progress: 102.0 %
Current progress: 102.0 %
Current progress: 102.0 %
Current progress: 102.0 %
Current progress: 102.0 %
Current progress: 102.0 %
Current progress: 103.0 %
Current progress: 103.0 %
Current progress: 103.0 %
Current progress: 103.0 %
Current progress: 103.0 %
Current progress: 103.0 %
Current progress: 103.0 %
Current progress: 103.0 %
Current progress: 103.0 %
Current progress: 103.0 %
Current progress: 104.0 %
Current progress: 104.0 %
Current progress: 104.0 %
Current progress: 104.0 %
Current progress: 104.0 %
Current progress: 104.0 %
Current progress: 104.0 %
Current progress: 104.0 %
Current progress: 104.0 %
Current progress: 105.0 %
Current progress: 105.0 %
Current progress: 105.0 %
Current progress: 105.0 %
Current prog

Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 137.0 %
Current progress: 140.0 %
Current progress: 140.0 %
Current progress: 140.0 %
Current progress: 140.0 %
Current progress: 140.0 %
Current progress: 140.0 %
Current progress: 140.0 %
Current progress: 140.0 %
Current progress: 140.0 %
Current progress: 140.0 %
Current prog

...

Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1812.0 %
Current progress: 1813.0 %
Current progress: 1813.0 %
Current progress: 1813.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
Current progress: 1814.0 %
C

Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1844.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1845.0 %
Current progress: 1846.0 %
Current progress: 1846.0 %
Current progress: 1846.0 %
Current progress: 1846.0 %
Current progress: 1846.0 %
Current progress: 1846.0 %
Current progress: 1846.0 %
Current progress: 1846.0 %
C

Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1876.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
Current progress: 1877.0 %
C

Current progress: 1909.0 %
Current progress: 1909.0 %
Current progress: 1909.0 %
Current progress: 1910.0 %
Current progress: 1910.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1912.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
Current progress: 1913.0 %
C

Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1949.0 %
Current progress: 1951.0 %
Current progress: 1951.0 %
Current progress: 1951.0 %
Current progress: 1951.0 %
Current progress: 1951.0 %
Current progress: 1952.0 %
Current progress: 1952.0 %
Current progress: 1952.0 %
Current progress: 1952.0 %
Current progress: 1952.0 %
Current progress: 1952.0 %
Current progress: 1952.0 %
Current progress: 1952.0 %
Current progress: 1952.0 %
Current progress: 1953.0 %
Current progress: 1953.0 %
Current progress: 1953.0 %
Current progress: 1953.0 %
Current progress: 1953.0 %
Current progress: 1953.0 %
Current progress: 1953.0 %
Current progress: 1953.0 %
Current progress: 1953.0 %
Current progress: 1953.0 %
Current progress: 1954.0 %
Current progress: 1954.0 %
C

In [ ]:
https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&historical=true&searchTag=-3&state=CA&subject3=9&gradeType=1&gradeLevel=-1

In [139]:
r.text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="utf-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n<meta name="format-detection" content="telephone=no" />\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\n\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\n\t\t\n\t

In [140]:
soup = BeautifulSoup(r.content, 'html.parser')

In [142]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="telephone=no" name="format-detection"/>
  <title>
   Find a classroom to support | DonorsChoose
  </title>
  <meta content="DonorsChoose makes it easy for anyone to help students in need. Public school teachers from every corner of America post classroom project requests on our site, and you can give any amount to the project that most inspires you." name="description"/>
  <meta content="https://cdn.donorschoose.net/images/logo/donorschoose_org_social_1200x630.png" property="og:image"/>
  <meta content="https://cdn.donorschoose.net/images/logo/donorschoose_org_social_1200x630.png" property="og:image:secure_url"/>
  <meta content="630" property="og:image:height"/>
  <meta content="1200" property="og:image:width"/>
  <meta content="https://www.donorschoose.org/donors/search.html?historical=true&amp;searchTag=-3&amp;state=ME&amp;subject1=-1&

In [143]:
list(soup.children)

['\n',
 'html',
 '\n',
 <html lang="en">
 <head>
 <meta charset="utf-8"/>
 <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
 <meta content="telephone=no" name="format-detection"/>
 <title>Find a classroom to support | DonorsChoose</title>
 <meta content="DonorsChoose makes it easy for anyone to help students in need. Public school teachers from every corner of America post classroom project requests on our site, and you can give any amount to the project that most inspires you." name="description"/>
 <meta content="https://cdn.donorschoose.net/images/logo/donorschoose_org_social_1200x630.png" property="og:image"/>
 <meta content="https://cdn.donorschoose.net/images/logo/donorschoose_org_social_1200x630.png" property="og:image:secure_url"/>
 <meta content="630" property="og:image:height"/>
 <meta content="1200" property="og:image:width"/>
 <meta content="https://www.donorschoose.org/donors/search.html?historical=true&amp;searchTag=-3&amp;state=ME&amp;subject1=-1&amp;grade

In [144]:
[type(item) for item in list(soup.children)]

[bs4.element.NavigableString,
 bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [145]:
html = list(soup.children)[3]

In [146]:
list(html.children)

['\n',
 <head>
 <meta charset="utf-8"/>
 <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
 <meta content="telephone=no" name="format-detection"/>
 <title>Find a classroom to support | DonorsChoose</title>
 <meta content="DonorsChoose makes it easy for anyone to help students in need. Public school teachers from every corner of America post classroom project requests on our site, and you can give any amount to the project that most inspires you." name="description"/>
 <meta content="https://cdn.donorschoose.net/images/logo/donorschoose_org_social_1200x630.png" property="og:image"/>
 <meta content="https://cdn.donorschoose.net/images/logo/donorschoose_org_social_1200x630.png" property="og:image:secure_url"/>
 <meta content="630" property="og:image:height"/>
 <meta content="1200" property="og:image:width"/>
 <meta content="https://www.donorschoose.org/donors/search.html?historical=true&amp;searchTag=-3&amp;state=ME&amp;subject1=-1&amp;gradeType=4&amp;schoolType=16" property

In [91]:
r.headers['content-type']

'text/html;charset=UTF-8'

In [96]:
from urllib.request import urlopen

In [104]:
soup = BeautifulSoup(urlopen(x)) # may use `cchardet` for speed

In [111]:
print(soup.get_text())






Find a classroom to support | DonorsChoose










































	You're on track to get doubled donations (and unlock a reward for the colleague who referred you). Keep up the great work!







Check out







			Take credit for your charitable giving! Check out your tax receipts






In a rush to make a major tax-deductible gift before the year's end? Purchase account credits and choose projects later!







Time is running out! Make your  tax-deductible gift before midnight on December 31.







TODAY ONLY: When you support a classroom, your donation will receive a 50% match from an anonymous supporter.



Donate



To use your $50 gift card credits, find a project to fund and we'll automatically apply your credits at checkout.
Find a classroom project




Your promo code has been successfully applied and will be reflected in checkout.





					Check your email to verify your account and get started on your first project. Didn't receive the email?






In [101]:
for link in soup.find_all('a'):
    print(link.get('href'))

https://secure.donorschoose.org/donors/givingCart.html
#
https://secure.donorschoose.org/donors/giving_history.html
#
https://www.donorschoose.org/credits
#
#
https://secure.donorschoose.org/credits
https://www.donorschoose.org/donors/search.html
https://secure.donorschoose.org/teacher/confirmEmail.html
#
#
https://www.donorschoose.org/teachers
https://www.donorschoose.org/
#mainBody
https://www.donorschoose.org/
https://www.donorschoose.org/donors/search.html
https://www.donorschoose.org/about
https://help.donorschoose.org/
https://www.donorschoose.org/about/accessibility.html
https://secure.donorschoose.org/common/signin.html?pageFromType=
https://secure.donorschoose.org/common/myaccount.html
#
https://secure.donorschoose.org/donors/forgot_password.html
https://www.donorschoose.org/donors/search.html?distanceLearning=only
None
https://www.donorschoose.org/donors/search.html?subject5=-5
https://www.donorschoose.org/donors/search.html?subject2=-2
https://www.donorschoose.org/donors/sea

In [102]:
print(soup.get_text())






Find a classroom to support | DonorsChoose










































	You're on track to get doubled donations (and unlock a reward for the colleague who referred you). Keep up the great work!







Check out







			Take credit for your charitable giving! Check out your tax receipts






In a rush to make a major tax-deductible gift before the year's end? Purchase account credits and choose projects later!







Time is running out! Make your  tax-deductible gift before midnight on December 31.







TODAY ONLY: When you support a classroom, your donation will receive a 50% match from an anonymous supporter.



Donate



To use your $50 gift card credits, find a project to fund and we'll automatically apply your credits at checkout.
Find a classroom project




Your promo code has been successfully applied and will be reflected in checkout.





					Check your email to verify your account and get started on your first project. Didn't receive the email?






In [82]:
#olderR = r

In [55]:
prog_count = 0
iteration = 0
out_file = "/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_"

for x in tenrandos:
#for x in search_urls:  #1674610
        
    page_to_use = x
    
    #Make the request
    try:
        r = requests.get(page_to_use) #send request to specific html address
    except:
        #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
        continue #if we have an exception, *continue* will skip the rest of the iteration
    
    try:
        data_dict = json.loads(r.text) #load the text as a dictionary
    except: #catch *all* exceptions
        #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
        continue #if we have an exception, *continue* will skip the rest of the iteration
    
    dicts_on_this_page = list(data_dict.values())
    proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
    proposal_list = dicts_on_this_page[proposal_ind]

    if len(proposal_list)>0:
        outframe = proposal_process(proposal_list,page_to_use)
        #print(outframe['teacherName'])
        pickle_out = open(out_file+("{0:07}".format(iteration))+'.pickle',"wb")
        pickle.dump(outframe, pickle_out)
        pickle_out.close()
        iteration += 1
        prog_count += 50
    
    print("Current progress:",np.round(iteration/(len(tenrandos))),"%")
    time.sleep(np.random.rand()*5)

Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %
Current progress: 0.0 %


In [56]:
#get all pickle files from folder into a list, 'picks'
picks = sorted(glob.glob("/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/*pickle")) 

In [57]:
#this will create a master list with the same # of elements as projects
basic_list = []
for pf in picks:
    print(pf)
    basic_list.append(pickle.load(open(pf,"rb")))
    #basic_list = basic_list+(pickle.load(open(picks[0],"rb")))

/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_0000000.pickle


In [58]:
bigframe = pd.concat(basic_list)

In [59]:
bigframe

,id,proposalURL,fundURL,imageURL,retinaImageURL,thumbImageURL,fulfillmentTrailer,percentFunded,numDonors,costToComplete,...,latitude,longitude,expirationDate,expirationTime,fundingStatus,shortDescription,matchingfund,grade_level,poverty_level,r_path
0,4952604,https://www.donorschoose.org/project/writers-o...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u66...,https://www.donorschoose.org/teacher/photo/u66...,https://www.donorschoose.org/teacher/photo/u66...,Help me give my students the basic needs to ac...,0,0,776.76,...,40.705326080322300,-111.957992553711000,2020-09-28,1601265600000,needs funding,My students are the most dedicated to their le...,no,Grades PreK-2,HIGHEST,https://www.donorschoose.org/common/json_feed....
0,4905847,https://www.donorschoose.org/project/learning-...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u31...,https://www.donorschoose.org/teacher/photo/u31...,https://www.donorschoose.org/teacher/photo/u31...,"Help me give my students supplies (pencils, no...",44,6,173.36,...,40.878350000000000,-111.900165000000000,2020-09-05,1599278400000,needs funding,My students are eager learners that enjoy the ...,no,Grades PreK-2,LOW,https://www.donorschoose.org/common/json_feed....
0,4961815,https://www.donorschoose.org/project/multicult...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u21...,https://www.donorschoose.org/teacher/photo/u21...,https://www.donorschoose.org/teacher/photo/u21...,Help me give my students a multicultural book ...,0,0,599.48,...,40.705326080322300,-111.957992553711000,2020-10-04,1601784000000,needs funding,"Rambunctious, energetic, wiggly, noisy, curiou...",no,Grades PreK-2,HIGHEST,https://www.donorschoose.org/common/json_feed....
0,4820880,https://www.donorschoose.org/project/wibble-wo...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u31...,https://www.donorschoose.org/teacher/photo/u31...,https://www.donorschoose.org/teacher/photo/u31...,"Help me give my students peanut bouncers, cube...",10,2,332.21,...,40.592316000000000,-111.925941000000000,2020-07-01,1593576000000,needs funding,My classroom has ten wonderful students with s...,no,Grades PreK-2,MODERATE,https://www.donorschoose.org/common/json_feed....
0,4820914,https://www.donorschoose.org/project/one-by-on...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u31...,https://www.donorschoose.org/teacher/photo/u31...,https://www.donorschoose.org/teacher/photo/u31...,Help me give my students materials that have b...,7,2,447.99,...,40.592316000000000,-111.925941000000000,2020-07-01,1593576000000,needs funding,My classroom has ten wonderful students with s...,no,Grades PreK-2,MODERATE,https://www.donorschoose.org/common/json_feed....
0,4905460,https://www.donorschoose.org/project/new-class...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,Help me give my students the chance to start a...,29,6,599.98,...,40.520959000000000,-111.957543000000000,2020-09-05,1599278400000,needs funding,Our students are a cultural mix of hispanic no...,no,Grades PreK-2,LOW,https://www.donorschoose.org/common/json_feed....
0,4809374,https://www.donorschoose.org/project/loved-sec...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,Help me give my students class sets of loved b...,3,1,945.76,...,41.075082000000000,-112.039996000000000,2020-06-26,1593144000000,needs funding,My students come from a variety of backgrounds...,no,Grades PreK-2,LOW,https://www.donorschoose.org/common/json_feed....


In [67]:
pd.options.display.max_colwidth = 1000

In [68]:
paths = bigframe['r_path']

In [69]:
print(str(paths[0]))

0    https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&state=UT&gradeType=1&gradeLevel=2&schoolType=5https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&state=UT&gradeType=1&gradeLevel=2&schoolType=5https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&state=UT&gradeType=1&gradeLevel=2&schoolType=5https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&state=UT&gradeType=1&gradeLevel=2&schoolType=5https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&state=UT&gradeType=1&gradeLevel=2&schoolType=5https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&state=UT&gradeType=1&gradeLevel=2&schoolType=5https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&state=UT&gradeType=1&gradeLevel=2&schoolType=5
0    ht

In [ ]:
bigframe['proposalURL'].value_counts()

In [ ]:
checkdf = pd.DataFrame(project_IDs,columns =['proj_id'])
checkdf
pd.set_option("max_rows", None) #undo by resetting --- pd.reset_option(“max_rows”)
checkdf

In [ ]:
checkdf['proj_id'].value_counts()

In [ ]:
basic_list[0]

In [ ]:
bewild = pd.DataFrame.from_dict(lookat)

In [ ]:
import json
from bs4 import BeautifulSoup

In [ ]:
import scrapy
#modified scrapy settings here:
#/home/russell/anaconda3/envs/insight/lib/python3.8/site-packages/scrapy/settings
#to include the user agents described here: https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/

In [ ]:
df = pd.read_csv("/home/russell/Downloads/opendata_projects.csv", thousands = ',')
pd.set_option('display.max_columns', None)
#pd.set_option("max_rows", None) #undo by resetting --- pd.reset_option(“max_rows”)bb

In [ ]:
current = "https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=100&index=0"
historical = "https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=40&historical=true&index=0"


In [ ]:
r = requests.get(historical)
data_dict = json.loads(r.text)

In [ ]:
print(data_dict)

In [ ]:
for key in data_dict.items():
    print (key)

In [ ]:
dicts_on_this_page = list(data_dict.values())

proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #

proposal_list = dicts_on_this_page[proposal_ind]
#dicts_on_this_page[proposal_ind]
#proposal_ind = which(data.keys()=='proposals')
#print(proposal_ind)
#print(proposals_on_this_page[7])

In [ ]:
proposal_list[0]

In [ ]:
first_proposal = proposal_list[0] #returns dictionary of first proposal items

In [ ]:
type(first_proposal)

In [ ]:
for key in first_proposal.items():
    print (key)

In [ ]:
print(soup)

# _DRAW OUT SCREENS and FUNCTIONALITY GOALS FOR END OF WEEK_



## MONDAY
    Well scoped, clearly-defined problem + some data
## 	TUESDAY
    SQL[organized data] 
## 	WED 
    analytics/working algo/some results ----> (python linked to sql data)
## 	THUR – SQL mapped to PYTHON connected to FLASK (or something)


# GOOD Qs to ASK SELF AND OTHERS
#### 	-What’s actionable about your product?
#### 	-Did you try other models?
#### 	-Is this better than random?
#### 	-Is this better than the simplest model?
#### 	-Why did you choose these inputs?
#### 	-How did you validate this?
#### 	-What are your metrics for success?
#### 	-What are the assumptions of your model?
#### 	-How would you improve this project with more time?

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df['primary_focus_area'].value_counts()

In [ ]:
expired = df[(df['funding_status']=='expired')]

In [ ]:
df['_teacher_acctid'].value_counts()